In [1]:
# load model from 2.2 (last epoch)
from keras.models import load_model
import os
model_path = r'C:\Users\Carla Pugh\PycharmProjects\VideoSlicing\jupyter notebooks\model2_3_chkp'
filename = 'weights.51-1.41.hdf5'
fullpath = os.path.join(model_path,  filename)
model = load_model(fullpath)

In [2]:
# load test data
from keras_video import VideoFrameGenerator
import os
import glob
import keras

glob_pattern='../data_v3_model_2.2/test_data/{classname}/*.avi'

classes = ['suture_throws', 'hand_ties', 'thread_cuts']
classes.sort()

# Create video frame generator
# some global params
SIZE = (100, 100) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val = 0.000001,
    split_test=0.999999,
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class hand_ties, validation count: 0, test count: 42, train count: 1
class suture_throws, validation count: 0, test count: 240, train count: 1
class thread_cuts, validation count: 0, test count: 23, train count: 1
Total data: 3 classes for 3 files for train


In [3]:
test = train.get_test_generator()

Total data: 3 classes for 305 files for test


In [4]:
files=test.files

In [5]:
a = model.predict_generator(test)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [6]:
len(a)

304

In [7]:
a

array([[1.47635881e-02, 5.93228685e-03, 9.79304075e-01],
       [9.18441832e-01, 8.15745443e-05, 8.14765915e-02],
       [6.22011954e-04, 2.85665952e-02, 9.70811367e-01],
       [9.23265874e-01, 6.67376953e-05, 7.66674131e-02],
       [3.96243930e-01, 1.09287282e-03, 6.02663159e-01],
       [6.12726152e-01, 6.41039573e-04, 3.86632890e-01],
       [7.72786736e-01, 2.54987390e-04, 2.26958230e-01],
       [8.82299483e-01, 1.21684534e-04, 1.17578804e-01],
       [7.25655293e-04, 1.00911289e-01, 8.98363054e-01],
       [9.93632138e-01, 2.82032761e-06, 6.36503613e-03],
       [4.78464784e-03, 1.46744037e-02, 9.80540931e-01],
       [1.18665859e-01, 3.42143443e-03, 8.77912700e-01],
       [9.18070138e-01, 5.86929164e-05, 8.18712339e-02],
       [5.79033434e-01, 5.99980354e-04, 4.20366615e-01],
       [9.92228687e-01, 3.94547487e-06, 7.76741561e-03],
       [6.18446385e-03, 4.22440609e-03, 9.89591181e-01],
       [8.48625496e-04, 8.71812776e-02, 9.11970079e-01],
       [9.91523147e-01, 3.74744

In [8]:
classes

['hand_ties', 'suture_throws', 'thread_cuts']

In [9]:
classes = ['hand_ties', 'suture_throws', 'thread_cuts']
def predict_maneuver(probs):
    mx = -1
    mx_index = -1
    for i in range(len(probs)):
        if probs[i] > mx:
            mx = probs[i]
            mx_index = i
    return classes[mx_index]

In [10]:
# naive way: I am going to assume that the last 5 files are not predicted

def get_name(fpath):
    fname = fpath.split('\\')[1][:-4]
    pre = fname.split('_')[0]
    for name in classes:
        if pre in name:
            return name
        
predictions = []
y_true = []
y_pred = []
for i in range(len(a)):
    predictions.append(
    {
        'correct_label': get_name(files[i]),
        #'correct_label': files[i],
        'prediction': predict_maneuver(a[i])
    })    
    y_true.append(get_name(files[i]))
    y_pred.append(predict_maneuver(a[i]))

In [11]:
predictions

[{'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'suture_throws'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'pre

In [11]:
from sklearn.metrics import accuracy_score

accuracy: 0.22697368421052633


In [15]:
thred_pred_cnt = 0
actual_thred_cnt = 0
for i in range(len(predictions)):
    if predictions[i]['correct_label'] == 'thread_cuts':
        actual_thred_cnt += 1
    if predictions[i]['prediction'] == 'thread_cuts':
        thred_pred_cnt += 1

print('accuracy: {}'.format(accuracy_score(y_true, y_pred)))
print('fraction of predictions that are thread cuts: {}'.format(thred_pred_cnt/len(predictions)))
print('fraction of inputs that are thread cuts: {}'.format(actual_thred_cnt/len(predictions)))

accuracy: 0.22697368421052633
fraction of predictions that are thread cuts: 0.7467105263157895
fraction of inputs that are thread cuts: 0.07236842105263158
